## Definicion del contexto

El contexto con el que quiero trabajar mi tarea es un proyecto personal en el que me encuentro trabajando el cual corresponde a un bot de trading algorítmico basado en señales. Éste toma estrategias de inversion usando indicadores técnicos y genera señales de compra y venta que ejecuta en los mercados a los que se encuentra conectado mediante API rest o recibiendo datos de manera ágil usando websockets de algunos comercios.

![TradingBot](TradingBot.png)

## Identificacion de requerimientos asociados al caso de uso:

Dentro de los posibles requerimientos asociados al caso de uso tenemos:
(Click para abrir cada item)
<details>
<summary>Soporte para múltiples estrategias de trading</summary>

- **Problema:** El bot debe poder operar con diferentes estrategias (como medias móviles, RSI, MACD), sin que el código tenga que modificarse cada vez que se agrega una nueva estrategia.  
- **Patrón propuesto:** Factory Method  
- **Justificación:** Permite la creación dinámica de estrategias de trading sin acoplar la lógica del bot a una implementación específica, facilitando la extensión y el mantenimiento.  
</details>

<details>
<summary>Conexión con múltiples exchanges</summary>

- **Problema:** El bot debe integrarse y conectarse a diferentes exchanges, cada uno con sus propias APIs y protocolos (por ejemplo, Binance, Coinbase, KuCoin).  
- **Patrón propuesto:** Abstract Factory  
- **Justificación:** Facilita la creación de familias de objetos relacionados (cliente API, autenticador, parser de datos) sin exponer las clases concretas, garantizando la compatibilidad y el desacoplamiento entre los diferentes exchanges.  
</details>

<details>
<summary>Configuración flexible del bot</summary>

- **Problema:** El bot requiere una configuración compleja y flexible que incluya parámetros como selección de estrategias, activos, límites de riesgo, intervalos temporales y otros ajustes operativos.  
- **Patrón propuesto:** Builder  
- **Justificación:** Permite construir configuraciones paso a paso de manera legible y modular, evitando constructores con demasiados parámetros y facilitando la personalización y extensión futura.  
</details>

<details>
<summary>Gestión de instancias únicas</summary>

- **Problema:** El bot necesita garantizar que ciertos componentes críticos (por ejemplo, el logger, gestor de configuración o conexión a base de datos) existan como una única instancia en toda la aplicación.  
- **Patrón propuesto:** Singleton  
- **Justificación:** Asegura que solo exista una instancia para componentes compartidos, lo que evita problemas de sincronización, duplicación de recursos y asegura la consistencia en el funcionamiento del sistema.  
</details>

<details>
<summary>Duplicación eficiente de configuraciones</summary>

- **Problema:** El bot requiere clonar objetos complejos que comparten configuraciones similares, como plantillas de órdenes o configuraciones de estrategias, para luego ajustarlas rápidamente a diferentes condiciones de mercado.  
- **Patrón propuesto:** Prototype  
- **Justificación:** Permite la clonación de objetos existentes y su posterior modificación sin necesidad de construirlos desde cero, ahorrando tiempo y recursos en la creación de objetos similares.  
</details>

<details>
<summary>Optimización del uso de recursos</summary>

- **Problema:** El bot debe gestionar de manera eficiente recursos costosos en su instanciación, como conexiones a APIs o módulos de análisis en tiempo real, para evitar sobrecargas y mejorar el rendimiento.  
- **Patrón propuesto:** Object Pool  
- **Justificación:** Facilita la reutilización de instancias ya creadas en lugar de generar nuevas constantemente, reduciendo la latencia y optimizando el uso de recursos.  
</details>

## Contexto integrado en la tarea:

Para la actividad propuesta resolveremos los siguientes:

- **Soporte para múltiples estrategias**  ya que el bot debe reaccionar a diferentes condiciones de mercado. Utilizando el Factory Method, el bot puede crear instancias de estrategias de trading de forma dinámica, permitiendo agregar o modificar estrategias sin afectar el comportamiento general del sistema.

- **Conexión con múltiples exchanges** es importante para diversificar la operativa y aprovechar las oportunidades de inversion en distintos mercados. Con el Abstract Factory, se pueden generar conjuntos de objetos relacionados (como clientes API y módulos de autenticación) para cada exchange, manteniendo una interfaz común y desacoplada.

- **Configuración flexible del bot** es necesaria para adaptarse a distintas condiciones operativas y perfiles de riesgo. El patrón Builder permite construir configuraciones de manera escalonada, modular y basada en plantillas base, facilitando la personalización y el mantenimiento del sistema conforme evoluciona la expansion del bot.

Esta combinación de problemas y patrones permite que el bot se adapte de forma ágil a cambios en el mercado, incorpore nuevas estrategias y se conecte de manera práctica a múltiples comercios.

## Impementacion **Soporte para múltiples estrategias**

![FactoryUML](Factory.svg)

In [2]:
from abc import ABC, abstractmethod

# Interfaz de la estrategia
class Strategy(ABC):
    @abstractmethod
    def execute(self):
        """Ejecutar la estrategia de trading."""
        pass

# Implementaciones De estrategias
class MovingAverageStrategy(Strategy):

    def __init__(self, ema_long_range, ema_short_range):
        self.ema_long_range: int= ema_long_range
        self.ema_short_range: int = ema_short_range

    def execute(self):
        print("Ejecutando estrategia de medias móviles long:{} short:{}"
              .format(self.ema_long_range, self.ema_short_range))

class RSIStrategy(Strategy):
    def __init__(self, rsi_range,upper_limit, lower_limit):
        self.rsi_range: int = rsi_range
        self.upper_limit: float = upper_limit
        self.lower_limit: float = lower_limit

    def execute(self):
        print("Ejecutando estrategia RSI range:{} upper:{} lower:{}"
              .format(self.rsi_range, self.upper_limit, self.lower_limit))

class MACDStrategy(Strategy):
    def __init__(self, ema_long_range, ema_short_range, signal_range):
        self.ema_long_range: int = ema_long_range
        self.ema_short_range: int = ema_short_range
        self.signal_range: int = signal_range

    def execute(self):
        print("Ejecutando estrategia MACD long:{} short:{} signal:{}"
              .format(self.ema_long_range, self.ema_short_range, self.signal_range))

# Fábrica Abstracta (Creador Base)
class StrategyCreator(ABC):
    @abstractmethod
    def create_strategy(self) -> Strategy:
        """Método que debe implementar cada creador concreto"""
        pass

# Fábricas Concretas
class MovingAverageCreator(StrategyCreator):
    def create_strategy(self,**kwargs) -> MovingAverageStrategy:
        return MovingAverageStrategy(**kwargs)

class RSICreator(StrategyCreator):
    def create_strategy(self,**kwargs) -> RSIStrategy:
        return RSIStrategy(**kwargs)

class MACDCreator(StrategyCreator):
    def create_strategy(self,**kwargs) -> MACDStrategy:
        return MACDStrategy(**kwargs)


# Uso
if __name__ == "__main__":
    strategy1 = MovingAverageCreator().create_strategy(ema_long_range=50, ema_short_range=20)
    strategy2 = RSICreator().create_strategy(rsi_range=14, upper_limit=70, lower_limit=30)
    strategy3 = MACDCreator().create_strategy(ema_long_range=26, ema_short_range=12, signal_range=9)
    
    strategy1.execute()
    strategy2.execute()
    strategy3.execute()
    


Ejecutando estrategia de medias móviles long:50 short:20
Ejecutando estrategia RSI range:14 upper:70 lower:30
Ejecutando estrategia MACD long:26 short:12 signal:9


## Impementacion **Conexión con múltiples exchanges**

AQUI VA EL UML

In [10]:

# Interfaces para componentes de exchange
class ExchangeAPI(ABC):
    @abstractmethod
    def connect(self):
        """Conecta con el exchange."""
        pass

class Authentication(ABC):
    @abstractmethod
    def authenticate(self):
        """Realiza la autenticación en el exchange."""
        pass

# Implementaciones para Binance
class BinanceAPI(ExchangeAPI):
    def connect(self):
        print("Conectando a Binance")
        pass

class BinanceAuth(Authentication):
    def authenticate(self):
        print("Autenticando en Binance")
        pass

# Implementaciones para Coinbase
class CoinbaseAPI(ExchangeAPI):
    def connect(self):
        print("Conectando a Coinbase")
        pass

class CoinbaseAuth(Authentication):
    def authenticate(self):
        print("Autenticando en Coinbase")
        pass

# Fabricas abstractas para crear componentes relacionados de un exchange
class ExchangeFactory(ABC):
    @abstractmethod
    def create_api(self) -> ExchangeAPI:
        """Crea una instancia de la API del exchange."""
        pass

    @abstractmethod
    def create_auth(self) -> Authentication:
        """Crea una instancia de la autenticación del exchange."""
        pass

class BinanceFactory(ExchangeFactory):
    def create_api(self) -> ExchangeAPI:
        return BinanceAPI()

    def create_auth(self) -> Authentication:
        return BinanceAuth()

class CoinbaseFactory(ExchangeFactory):
    def create_api(self) -> ExchangeAPI:
        return CoinbaseAPI()

    def create_auth(self) -> Authentication:
        return CoinbaseAuth()
    
# Cliente de uso
def ExchangeClient(factory: ExchangeFactory):
    api = factory.create_api()
    auth = factory.create_auth()

    return api, auth

# Uso
if __name__ == "__main__":
    api,auth = ExchangeClient(BinanceFactory())
    api.connect()
    auth.authenticate()


Conectando a Binance
Autenticando en Binance


## Implementacion **Configuración flexible del bot**

AQUI VA EL UML

In [11]:
class BotConfig:
    def __init__(self):
        self.strategy = None
        self.exchange_factory = None
        self.risk_limit = None
        self.assets = None

    def __str__(self):
        return (f"BotConfig(strategy={self.strategy.__class__.__name__ if self.strategy else None}, "
                f"exchange_factory={self.exchange_factory.__class__.__name__ if self.exchange_factory else None}, "
                f"risk_limit={self.risk_limit}, assets={self.assets})")

class BotConfigBuilder:
    def __init__(self):
        self._strategy = None
        self._exchange_factory = None
        self._risk_limit = 0.0
        self._assets = []

    def set_strategy(self, strategy: Strategy):
        self._strategy = strategy
        return self

    def set_exchange_factory(self, exchange_factory: ExchangeFactory):
        self._exchange_factory = exchange_factory
        return self

    def set_risk_limit(self, risk_limit: float):
        self._risk_limit = risk_limit
        return self

    def set_assets(self, assets: list):
        self._assets = assets
        return self

    def build(self) -> BotConfig:

        config = BotConfig()
        config.strategy = self._strategy
        config.exchange_factory = self._exchange_factory
        config.risk_limit = self._risk_limit
        config.assets = self._assets 
        return config
        

In [12]:
if __name__ == "__main__":
    # Crear una estrategia utilizando el Factory Method
    strategy = RSICreator().get_strategy()
    
    # Seleccionar la factoría para Binance (con Abstract Factory)
    exchange_factory = BinanceFactory()
    
    # Construir la configuración del bot utilizando el Builder Pattern
    config_builder = BotConfigBuilder()
    bot_config = (config_builder
                .set_strategy(strategy)
                .set_exchange_factory(exchange_factory)
                .set_risk_limit(0.05)
                .set_assets(["BTC", "ETH"])
                .build())
    
    print(bot_config)

Creando estrategia: RSIStrategy
BotConfig(strategy=RSIStrategy, exchange_factory=BinanceFactory, risk_limit=0.05, assets=['BTC', 'ETH'])
